In [1]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

### Loading the instance

### Solver

In [2]:
from mclp import get_random_bin

def get_adjusted_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.adj_vol*boxes[box]
    return vol

def get_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.vol*boxes[box]
    return vol

def adjusted_swap(solution, n=2,  max_vol_accept=1.0, tolerance=0.1, verbose=True):

    bin_A = get_random_bin(s = solution)
    bin_B = get_random_bin(s = solution, b = bin_A)

    iniA = bin_A.adj_vol
    iniB = bin_B.adj_vol
    var_ini = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2

    nA = int(random.randint(1, n))
    boxes_A = bin_A.pop_random_boxes(nA)
    adjvolA = get_adjusted_vol(boxes_A)

    nB = int(random.randint(0, n))
    boxes_B = bin_B.pop_random_boxes(nB)
    adjvolB = get_adjusted_vol(boxes_B)

    r = tolerance*random.random()
    accepted = False

    if adjvolA > adjvolB and bin_B.adj_vol + adjvolA <= max_vol_accept + r:
        accepted = True
    elif adjvolB > adjvolA and  bin_A.adj_vol + adjvolB <= max_vol_accept + r:
        accepted = True

    if accepted:
        bin_A.insert_boxes(boxes_B)
        bin_B.insert_boxes(boxes_A)
    else: 
        return -10

    var_final = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
    var_diff = var_final - var_ini

    return var_diff

def get_sorted_vols(solution):
    volumens = []
    acum_volu = 0

    for s in solution:
        s_boxes = s.boxes
        vol_boxes = get_vol(s_boxes)
        acum_volu += get_vol(s_boxes)
        volumens.append(vol_boxes)

    volumens.sort()

    return volumens

""" 
    Obtiene los bins ordenados por sus volumenes de menor a mayor
    [1,2,4,11,1,23,5,123] ---(sort) --> [1, 1, 2, 4, 5, 11, 23, 123]
    tip: Agregar reverse igual True para ordenarlas de mayor a menor
"""
def get_sorted_bins(solution):

    list_solution = list(solution)
    list_solution.sort(key=lambda x: x.vol)

    return list_solution


In [3]:
from bsg import (load_BRinstance,load_BRKGAinstance, load_LargeInstance, 
                load_productInstance, load_instances_elhedhli)
from mclp import generate_candidate_solution
import numpy as np

import statistics as st
import time

total_ejecution = 1         # Total de ejecuciones por instancias
init_instance = 1           # Instancia desde donde comenzar
total_inst = 6              # Total de instancias para la clase
total_class = 5             # Total de clases
t_boxes = 1000              # Cantidad de cajas

r_param = 2                 # Parametro de llenado del contenedor
min_fr = 0.99               # minimo porcentaje ocupado para considerar la generación de un bloque
max_bl = 10000              # maxima cantidad de bloques a generar por bsg
bsg_time = 20               # Tiempo de BSG
extra_args = '--max_bl={} --min_fr={}'.format(max_bl, min_fr)

for i in range(1,total_class):

    #print(f"Class {i}")
    for i_inst in range(init_instance, total_inst):

        file_name_column = f"../benchs/Instance_CG/{t_boxes}/bin_pack_instance_i({i_inst})_c({i}).txt"

        L,W,H,boxes,id2box = load_instances_elhedhli(filename=file_name_column)

        boxes=boxes.copy()

        solutions = []
        times = []
        less_volu = []
        _boxes=boxes.copy()

        vol_boxes = 0
        for key,value in id2box.items():
            vol_boxes += value.vol

        Vmax = L*W*H            # Volumen del contenedor

        for eje in range(total_ejecution):

            initial_time = time.time()
            boxes =_boxes.copy()

            #Generate solution
            init_solution = generate_candidate_solution(ssh, L, W, H, boxes, id2box, r_param=r_param, bsg_time=bsg_time, extra_args=extra_args)
            last_time = time.time()
            
            #Execution time
            execution_time = last_time - initial_time
            
            # Sort the vol of the minor vol bin
            bins_to_plot = get_sorted_vols(init_solution)
            eje_x = [x for x in range(0,len(bins_to_plot))]
            minor_vol = bins_to_plot[0]

            solutions.append(len(eje_x))
            less_volu.append(minor_vol)
            times.append(execution_time)

        best_solution = min(solutions)
        mean_sol = st.mean(solutions)

        #std_dev_sol = st.stdev(solutions)
        #std_dev_vol = st.stdev(less_volu)
        mean_vol = st.mean(less_volu)
        mean_time = st.mean(times)


        usage_percentage = mean_vol/Vmax
        mean_time = str(mean_time).replace('.',',')
        usage_percentage = str(usage_percentage).replace('.', ',')
        
        # instancia promedio_solucion porcentage_de_uso tiempo_de_ejecucion_promedio
        print(f"{i_inst} {mean_sol} {usage_percentage} {mean_time} ")

1 9 0,03902507907542579 161,6608428955078 
2 8 0,6921706645377129 141,5263466835022 
3 8 0,6856490328467153 141,65096640586853 
4 8 0,7684911430454177 142,84494304656982 
5 8 0,6418576855231144 141,425213098526 
1 8 0,44503313767234387 141,4945330619812 
2 9 0,009915389294403894 161,63144421577454 
3 8 0,6835245123682077 141,49238538742065 
4 8 0,5229630885036496 141,65925979614258 
5 8 0,7539063528994323 143,17599534988403 
1 7 0,19390773215733983 121,43255972862244 
2 7 0,2519700699513382 121,65937662124634 
3 7 0,35549441605839416 121,42834854125977 
4 7 0,13085901814679643 121,36802625656128 
5 7 0,23432472678426602 121,35641622543335 
1 6 0,769756349351176 101,32780241966248 
2 6 0,6781454835766423 101,49286794662476 
3 6 0,7255402858880778 101,35558223724365 
4 6 0,5970674614760746 101,45982527732849 
5 6 0,7277841098945661 101,47082281112671 
